In [1]:
import joblib

dataset = joblib.load('Random Forest.pkl')
dataset

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

In [2]:
# import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,root_mean_squared_error,r2_score
import statsmodels.api as sm

In [3]:
dk = pd.read_csv('test.csv')   
dk.head()

,credit_score,income,loan_amount,loan_term,interest_rate,debt_to_income_ratio,employment_years,savings_balance,age,default_risk_score
0,340,31745,8983,48,10.12,18.33,11,91417,56,8342.948454
1,439,115081,42051,60,3.33,31.77,6,17127,51,1710.293506
2,619,35313,6619,36,9.06,24.29,20,46761,42,8744.629774
3,664,72269,19226,48,4.29,12.50,26,64340,36,6191.771456
4,564,46292,30476,48,11.15,40.55,20,93831,50,3990.722206


In [4]:
X = dk.drop('default_risk_score', axis=1)
y = dk['default_risk_score']

In [5]:
#split into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Scaling variables
scaled_columns = ['loan_term','interest_rate', 'debt_to_income_ratio','employment_years','age']
scaler = StandardScaler()
dk[scaled_columns] = scaler.fit_transform(dk[scaled_columns])
dk.head()

,credit_score,income,loan_amount,loan_term,interest_rate,debt_to_income_ratio,employment_years,savings_balance,age,default_risk_score
0,340,31745,8983,0.647675,0.494890,-0.967770,-0.615437,91417,0.947036,8342.948454
1,439,115081,42051,1.336692,-1.653229,0.249744,-1.125750,17127,0.603409,1710.293506
2,619,35313,6619,-0.041341,0.159543,-0.427860,0.303126,46761,-0.015120,8744.629774
3,664,72269,19226,0.647675,-1.349519,-1.495903,0.915501,64340,-0.427472,6191.771456
4,564,46292,30476,0.647675,0.820746,1.045115,0.303126,93831,0.534684,3990.722206


In [7]:
# log transformation
log_columns = ['credit_score', 'loan_amount','savings_balance','default_risk_score']
dk[log_columns] = dk[log_columns].apply(lambda x:np.log1p(x))
dk.head()

,credit_score,income,loan_amount,loan_term,interest_rate,debt_to_income_ratio,employment_years,savings_balance,age,default_risk_score
0,5.831882,31745,9.103200,0.647675,0.494890,-0.967770,-0.615437,11.423198,0.947036,9.029292
1,6.086775,115081,10.646662,1.336692,-1.653229,0.249744,-1.125750,9.748470,0.603409,7.445005
2,6.429719,35313,8.797851,-0.041341,0.159543,-0.427860,0.303126,10.752826,-0.015120,9.076309
3,6.499787,72269,9.864071,0.647675,-1.349519,-1.495903,0.915501,11.071952,-0.427472,8.731138
4,6.336826,46292,10.324728,0.647675,0.820746,1.045115,0.303126,11.449261,0.534684,8.291978


In [8]:
#split into training and testing
X_train, X_test, y_train,y_test = train_test_split(X,y, test_size = 0.2,random_state = 42)

In [9]:
# feature selection using RFE
from sklearn.feature_selection import RFE
# fit regression model
lr = LinearRegression()
# select top 5 features using RFE
rfe = RFE(estimator = lr, n_features_to_select = 6)
X_selected_rfe = rfe.fit_transform(X,y)
# get selected feature names
selected_features = X.columns[rfe.support_]
print(f"Selected_features : {selected_features}")

Selected_features : Index(['credit_score', 'loan_amount', 'interest_rate', 'debt_to_income_ratio',
       'employment_years', 'age'],
      dtype='object')


In [10]:
X = dk[selected_features]
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Make Predictions with the loaded model
y_pred = dataset.predict(X_test)

In [11]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Calculate metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)  # Square root of MSE
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print results
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"R-squared (R²): {r2:.4f}")

Mean Squared Error (MSE): 2462.2647
Root Mean Squared Error (RMSE): 49.6212
Mean Absolute Error (MAE): 41.8137
R-squared (R²): 0.9996
